In [10]:
!pip install korpora
!pip install jpype1 konlpy

In [11]:
import torch
import torch.nn as nn
import pandas as pretrained_models

import re
import string

from Korpora import Korpora
from konlpy.tag import Okt

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [12]:
# data 다운로드
corpus = Korpora.load('nsmc')


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\USER\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\USER

In [13]:
# text와 label 추출
all_text = corpus.get_all_texts()
all_label = corpus.get_all_labels()

# train, test set 생성
train = corpus.train
test = corpus.test

In [16]:
# 전처리 함수
def text_preprocessing(text, tokenizer):    
    text = text.lower() # 소문자화
    text = re.sub(f'[{string.punctuation}]', ' ', text) # 구두점 제거
    text = tokenizer.morphs(text, stem= True) # stem -> 어간 추출 여부
    
    return ' '.join(text)

In [17]:
text = "이 Movie 정말 재미 있어요....,!!!!!!. 꼭보세요.### 저기 있네요."
okt = Okt()
text_preprocessing(text, okt)

'이 movie 정말 재미 있다 꼭 보다 저기 있다'

In [18]:
! pip install tokenizers

In [19]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer

In [21]:
# 전처리한 text 저장 경로
corpus_path = 'datasets/nsmc_morphs.txt'

# 모든 원소를 하나의 string으로
with open(corpus_path, 'wt', encoding= 'utf-8') as fw:
    fw.write('\n'.join(all_text))

# hugginhface tokenizer lib 사용
# subword 방식(BPE)
vocab_size = 30000  # 최대 단어수
min_frequency = 5   # 최소 빈도수 조건

tk = Tokenizer(
    BPE(unk_token= '[UNK]') # unknown 토큰 - 모르는 토큰 표시
)
tk.pre_tokenizer = Whitespace() # 공백을 기준

trainer = BpeTrainer(
    vocab_size= vocab_size,
    min_frequency= min_frequency,
    special_tokens= ['[PAD', '[UNK]'], # 사용자가 등록하는 토큰들, 특정 의미를 가지는 토큰들을 추가
)

In [22]:
# 학습
tk.train([corpus_path], trainer= trainer)

# tokenizer 저장
tk.save('models.msmc_bpe.json')

# load_tk = Tokenizr.from_file(경로)

In [26]:
# tokenizer 조회
# 문자열 토큰 -> 정수 index, 없으면 None
print(tk.token_to_id('한층'))

# 정수 index -> 문자열 토큰, 없으면 None
print(tk.id_to_token(16959))
print(tk.id_to_token(30000))

print('num of total token', tk.get_vocab_size())

# 전체 token 조회 (token : id 형태의 dict)
# tk.get_vocab() 

16959
한층
None
num of total token 30000


In [28]:
# pytorch Dataset, DataLoader 생성
tk.token_to_id('[PAD]')

In [33]:
from torch.utils.data import Dataset, DataLoader

class NSMCDataset(Dataset):
    def __init__(self, text, label, max_length, tokenizer):
        self.text= text             # 전체 input
        self.label= label           # 전체 label
        self.max_length= max_length # text의 최대 토큰수
        self.tokenizer= tokenizer

    def __getitem__(self, idx):
        text= self.text[idx]                # 문장
        encode= self.tokenizer.encode(text) # 문자열 기반 토큰화
        text_tokens_id= encode.ids          # 

        PAD_TOKEN_ID= self.tokenizer.token_to_id('[PAD]')
        seq_len= len(text_tokens_id)        # idx번째 문서의 토큰 개수
        result= None

        if seq_len >= self.max_length:
            result= text_tokens_id[:self.max_length]
        else: # max_length보다 토큰수가 적으면 -> 뒤에 [PAD] 추가
            result= text_tokens_id + ([PAD_TOKEN_ID] * (self.max_length - seq_len))

        label= self.label[idx]

        return (torch.tensor(result, dtype= torch.int64),
                torch.tensor([label], dtype= torch.float32))
    
    def __len__(self):
        return len(self.text)

In [31]:
# Dataset 생성
okt = Okt()
train_texts = [text_preprocessing(txt, okt) for txt in corpus.train.texts]
test_texts = [text_preprocessing(txt, okt) for txt in corpus.test.texts]

In [36]:
MAX_LEN = 40

train_set = NSMCDataset(
    train_texts,
    corpus.train.labels,
    max_length= MAX_LEN,
    tokenizer= tk
)

test_set = NSMCDataset(test_texts, corpus.test.labels, max_length= MAX_LEN, tokenizer= tk)

In [37]:
# DataLoader
BATCH_SIZE = 64
train_loader = DataLoader(train_set, batch_size= BATCH_SIZE, shuffle= True, drop_last= True)
test_loader = DataLoader(test_set, batch_size= BATCH_SIZE)

In [ ]:
class NSMCModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers, dropout_rate= 0):
        super().__init__()